In [1]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/iADR')
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import pandas as pd
import time
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from keras import regularizers
from sklearn import preprocessing

Mounted at /content/drive


In [2]:
data1 = np.load('./data/gin_supervised_masking.npy')

y = pd.read_csv('./data/labels(2248).csv')
k_X_train,k_x_test,k_Y_train,k_y_test= train_test_split(data1,y,test_size=0.1,random_state=2021)

# model

In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.multiclass import OneVsRestClassifier
import joblib

In [4]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
import joblib

In [5]:
m1 = keras.metrics.BinaryAccuracy(name='accuracy',threshold=0.5)
m2 = keras.metrics.Precision(name='precision')
m3 = keras.metrics.Recall(name='recall')
m4 = keras.metrics.AUC(name='AUC',multi_label=True,num_labels=27,num_thresholds=498)
m5 = keras.metrics.AUC(name='AUPR',curve='PR',multi_label=True,num_labels=27,num_thresholds=498)

In [6]:
#SVM
results1=[]
results2=[]
results3=[]
results4=[]
results5=[]
# 运行计时部分
time_start = time.time()
print(data1.shape)

kfolder = KFold(n_splits=10,shuffle=True,random_state=2021)
for i, (tra_id,val_id) in enumerate(kfolder.split(k_X_train,k_Y_train)):
  print(f"***********fold-{i+1}***********")

  c_x_train = k_X_train[tra_id]
  c_y_train = k_Y_train.iloc[tra_id]

  c_x_valid = k_X_train[val_id]
  c_y_valid = k_Y_train.iloc[val_id]



  # scaler = preprocessing.StandardScaler().fit(c_x_train)
  # c_x_train = scaler.transform(c_x_train)

  
  # scaler = preprocessing.StandardScaler().fit(c_x_valid)
  # c_x_valid = scaler.transform(c_x_valid)



  svc = SVC(probability=True)
  model = OneVsRestClassifier(svc, n_jobs=1)
  model.fit(c_x_train, c_y_train)



  y_pre = model.predict_proba(c_x_valid)
  # nlabel = np.array(c_y_valid)
  # plabel = np.where(c_pre_valid >= 0.5,1,0)    
  # print(nlabel,nlabel.shape)
  # print(y_pre_valid)
  # print(plabel)

  # y_pre = model.predict(c_x_valid)
  # np.save('./data/masking_RF_pre'+str(i+1)+'.npy',y_pre)



  m1.update_state(c_y_valid,y_pre)
  m2.update_state(c_y_valid,y_pre)
  m3.update_state(c_y_valid,y_pre)
  m4.update_state(c_y_valid,y_pre)
  m5.update_state(c_y_valid,y_pre)



  


  # model.save('./model/model_masking_5fold'+str(i+1)+'.h5')
  # model.save('./model/attfp/RF_attfp_10fold'+str(i+1)+'.h5')

  # result1 = np.array(n1)
  results1.append(m1.result().numpy())
  results2.append(m2.result().numpy())
  results3.append(m3.result().numpy())
  results4.append(m4.result().numpy())
  results5.append(m5.result().numpy())


  print(f"{i+1}-fold cv val Accuracy score: {m1.result().numpy()}")
  print(f"{i+1}-fold cv val Precision score: {m2.result().numpy()}")
  print(f"{i+1}-fold cv val Recall score: {m3.result().numpy()}")
  print(f"{i+1}-fold cv val AUC score: {m4.result().numpy()}")
  print(f"{i+1}-fold cv val AUPR score: {m5.result().numpy()}")
  print("\n")
  # model.save_weights('./model/attfp/RF_attfp_10fold'+str(i+1)+'.h5')
  # 6、保存模型
  joblib.dump(model,'./model/masking/SVM_masking_10fold'+str(i+1)+'.pkl')





print("******************************************************************")
# print(np.array(results))

print(f"-Accuracy score_mean:{np.mean(results1)}")
print(f"-Accuracy score_std:{np.std(results1)}")

print(f"-Precision score_mean:{np.mean(results2)}")
print(f"-Precision score_std:{np.std(results2)}")

print(f"-Recall score_mean:{np.mean(results3)}")
print(f"-Recall score_std:{np.std(results3)}")

print(f"-AUC score_mean:{np.mean(results4)}")
print(f"-AUC score_std:{np.std(results4)}")

print(f"-AUPR score_mean:{np.mean(results5)}")
print(f"-AUPR score_std:{np.std(results5)}")






    
# 运行计时部分
time_end = time.time()
print(f"total running time: {(time_end - time_start)/60} minites")

(2248, 300)
***********fold-1***********
1-fold cv val Accuracy score: 0.7960225343704224
1-fold cv val Precision score: 0.7997256517410278
1-fold cv val Recall score: 0.8825310468673706
1-fold cv val AUC score: 0.724313497543335
1-fold cv val AUPR score: 0.7359602451324463


***********fold-2***********
2-fold cv val Accuracy score: 0.7920998930931091
2-fold cv val Precision score: 0.7826673984527588
2-fold cv val Recall score: 0.8965517282485962
2-fold cv val AUC score: 0.7171158790588379
2-fold cv val AUPR score: 0.7207376956939697


***********fold-3***********
3-fold cv val Accuracy score: 0.7949278950691223
3-fold cv val Precision score: 0.7936952114105225
3-fold cv val Recall score: 0.889822781085968
3-fold cv val AUC score: 0.7071180939674377
3-fold cv val AUPR score: 0.7194902896881104


***********fold-4***********
4-fold cv val Accuracy score: 0.7954057455062866
4-fold cv val Precision score: 0.7990456819534302
4-fold cv val Recall score: 0.8844122290611267
4-fold cv val AUC

In [7]:
# 7、模型的加载使用
model2 = joblib.load('./model/masking/SVM_masking_10fold1.pkl')
history  = model2.fit(k_X_train,k_Y_train)
y_pre = model2.predict_proba(k_x_test)
np.save('./model/masking/masking_svm_pre.npy',y_pre)

m1.update_state(k_y_test,y_pre)
m2.update_state(k_y_test,y_pre)
m3.update_state(k_y_test,y_pre)
m4.update_state(k_y_test,y_pre)
m5.update_state(k_y_test,y_pre)



print(f"Accuracy score: {m1.result().numpy()}")
print(f"Precision score: {m2.result().numpy()}")
print(f"Recall score: {m3.result().numpy()}")
print(f"AUC score: {m4.result().numpy()}")
print(f"AUPR score: {m5.result().numpy()}")

Accuracy score: 0.7955713868141174
Precision score: 0.8041699528694153
Recall score: 0.8797045350074768
AUC score: 0.6929309368133545
AUPR score: 0.724767804145813


In [5]:
results1=[]
results2=[]
results3=[]
results4=[]
results5=[]
# 运行计时部分
time_start = time.time()
print(data1.shape)

kfolder = KFold(n_splits=10,shuffle=True,random_state=2021)
for i, (tra_id,val_id) in enumerate(kfolder.split(k_X_train,k_Y_train)):
  print(f"***********fold-{i+1}***********")

  c_x_train = k_X_train[tra_id]
  c_y_train = k_Y_train.iloc[tra_id]

  c_x_valid = k_X_train[val_id]
  c_y_valid = k_Y_train.iloc[val_id]



  # scaler = preprocessing.StandardScaler().fit(c_x_train)
  # c_x_train = scaler.transform(c_x_train)

  
  # scaler = preprocessing.StandardScaler().fit(c_x_valid)
  # c_x_valid = scaler.transform(c_x_valid)



  rf = RandomForestClassifier(n_estimators=100)
  model = OneVsRestClassifier(rf, n_jobs=-1)
  history = model.fit(c_x_train, c_y_train)



  y_pre = model.predict_proba(c_x_valid)
  # nlabel = np.array(c_y_valid)
  # plabel = np.where(c_pre_valid >= 0.5,1,0)    
  # print(nlabel,nlabel.shape)
  # print(y_pre_valid)
  # print(plabel)

  # y_pre = model.predict(c_x_valid)
  # np.save('./data/masking_RF_pre'+str(i+1)+'.npy',y_pre)



  m1.update_state(c_y_valid,y_pre)
  m2.update_state(c_y_valid,y_pre)
  m3.update_state(c_y_valid,y_pre)
  m4.update_state(c_y_valid,y_pre)
  m5.update_state(c_y_valid,y_pre)



  


  # model.save('./model/model_masking_5fold'+str(i+1)+'.h5')
  # model.save('./model/attfp/RF_attfp_10fold'+str(i+1)+'.h5')

  # result1 = np.array(n1)
  results1.append(m1.result().numpy())
  results2.append(m2.result().numpy())
  results3.append(m3.result().numpy())
  results4.append(m4.result().numpy())
  results5.append(m5.result().numpy())


  print(f"{i+1}-fold cv val Accuracy score: {m1.result().numpy()}")
  print(f"{i+1}-fold cv val Precision score: {m2.result().numpy()}")
  print(f"{i+1}-fold cv val Recall score: {m3.result().numpy()}")
  print(f"{i+1}-fold cv val AUC score: {m4.result().numpy()}")
  print(f"{i+1}-fold cv val AUPR score: {m5.result().numpy()}")
  print("\n")
  # model.save_weights('./model/attfp/RF_attfp_10fold'+str(i+1)+'.h5')
  # 6、保存模型
  joblib.dump(model,'./model/masking/RF_masking_10fold'+str(i+1)+'.pkl')





print("******************************************************************")
# print(np.array(results))

print(f"-Accuracy score_mean:{np.mean(results1)}")
print(f"-Accuracy score_std:{np.std(results1)}")

print(f"-Precision score_mean:{np.mean(results2)}")
print(f"-Precision score_std:{np.std(results2)}")

print(f"-Recall score_mean:{np.mean(results3)}")
print(f"-Recall score_std:{np.std(results3)}")

print(f"-AUC score_mean:{np.mean(results4)}")
print(f"-AUC score_std:{np.std(results4)}")

print(f"-AUPR score_mean:{np.mean(results5)}")
print(f"-AUPR score_std:{np.std(results5)}")






    
# 运行计时部分
time_end = time.time()
print(f"total running time: {(time_end - time_start)/60} minites")

(2248, 300)
***********fold-1***********
1-fold cv val Accuracy score: 0.7967523336410522
1-fold cv val Precision score: 0.8085706233978271
1-fold cv val Recall score: 0.8683015704154968
1-fold cv val AUC score: 0.724037230014801
1-fold cv val AUPR score: 0.7490529417991638


***********fold-2***********
2-fold cv val Accuracy score: 0.796296238899231
2-fold cv val Precision score: 0.7950934171676636
2-fold cv val Recall score: 0.8820164203643799
2-fold cv val AUC score: 0.7219607830047607
2-fold cv val AUPR score: 0.731626033782959


***********fold-3***********
3-fold cv val Accuracy score: 0.796083390712738
3-fold cv val Precision score: 0.8020563125610352
3-fold cv val Recall score: 0.8768607378005981
3-fold cv val AUC score: 0.708625078201294
3-fold cv val AUPR score: 0.7291405200958252


***********fold-4***********
4-fold cv val Accuracy score: 0.7963647246360779
4-fold cv val Precision score: 0.8083152174949646
4-fold cv val Recall score: 0.8698506355285645
4-fold cv val AUC sc

In [6]:
# 7、模型的加载使用
model2 = joblib.load('./model/masking/RF_masking_10fold1.pkl')
history  = model2.fit(k_X_train,k_Y_train)
y_pre = model2.predict_proba(k_x_test)
np.save('./model/masking/masking_RF_pre.npy',y_pre)

m1.update_state(k_y_test,y_pre)
m2.update_state(k_y_test,y_pre)
m3.update_state(k_y_test,y_pre)
m4.update_state(k_y_test,y_pre)
m5.update_state(k_y_test,y_pre)



print(f"Accuracy score: {m1.result().numpy()}")
print(f"Precision score: {m2.result().numpy()}")
print(f"Recall score: {m3.result().numpy()}")
print(f"AUC score: {m4.result().numpy()}")
print(f"AUPR score: {m5.result().numpy()}")

Accuracy score: 0.7952747941017151
Precision score: 0.8127186894416809
Recall score: 0.8641216158866882
AUC score: 0.7038019895553589
AUPR score: 0.7313858270645142


# deppDDI

In [3]:
METRICS = [
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.AUC(name='auc',multi_label=True,num_labels=27,num_thresholds=498),
      keras.metrics.AUC(name='prc',curve='PR',multi_label=True,num_labels=27,num_thresholds=498), # precision-recall curve
]

def create_model_DDI(metrics=METRICS):
  model = Sequential([
    layers.Flatten(name="my_intermediate_layer"),
    layers.Dropout(0.5),
    layers.Dense(2048),
    layers.ReLU(),
    layers.Dropout(0.5),
    layers.Dense(2048),
    layers.ReLU(),
    layers.Dropout(0.5),
    layers.Dense(2048),
    layers.ReLU(),
    layers.Dropout(0.5),
    layers.Dense(2048),
    layers.ReLU(),
    layers.Dropout(0.5),
    layers.Dense(2048),
    layers.ReLU(),
    layers.Dropout(0.5),
    layers.Dense(2048),
    layers.ReLU(),
    layers.Dropout(0.5),
    layers.Dense(2048),
    layers.ReLU(),
    layers.Dropout(0.5),
    layers.Dense(2048),
    layers.ReLU(),
    layers.Dropout(0.5),
    layers.Dense(27, activation='sigmoid', name="my_last_layer"),
    ])
  return model

In [4]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau#回调函数
# # 定义回调函数：保存最优模型                     
checkpoint = ModelCheckpoint("./data/fp_cross-model.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             save_weights_only=False,
                             verbose=1)
# 定义回调函数：提前终止训练
earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0.0001, 
                          patience = 30,
                          verbose = 3,
                          mode = 'min',
                          restore_best_weights = True)
# 定义回调函数：学习率衰减
reduce_lr = ReduceLROnPlateau(monitor = 'val_loss',
                              factor = 0.2,
                              patience = 5,
                              verbose = 1,
                              min_delta = 0.0001)

# 将回调函数组织为回调列表
callbacks = [earlystop,reduce_lr]

In [5]:
results=[]
# 运行计时部分
time_start = time.time()
print(data1.shape)

kfolder = KFold(n_splits=10,shuffle=True,random_state=2021)
for i, (tra_id,val_id) in enumerate(kfolder.split(k_X_train,k_Y_train)):
  print(f"***********fold-{i+1}***********")

  c_x_train = k_X_train[tra_id]
  c_y_train = k_Y_train.iloc[tra_id]

  c_x_valid = k_X_train[val_id]
  c_y_valid = k_Y_train.iloc[val_id]



  model = create_model_DDI()





  opt = Adam(learning_rate=0.001)
  model.compile(loss='binary_crossentropy', optimizer=opt, metrics=METRICS)
    
  # training
  history = model.fit(c_x_train,c_y_train,epochs=300,batch_size=128,
                      validation_data=(c_x_valid, c_y_valid.values),callbacks=callbacks,)
  


  y_pre = model.predict(c_x_valid)
  # y_pre1 = model.predict(k_x_test)
  # np.save('./data/FP_DDI__pre'+str(i+1)+'.npy',y_pre)

  baseline_results = model.evaluate(c_x_valid, c_y_valid.values, verbose=0)
  for name, value in zip(model.metrics_names, baseline_results):
    print(name, ': ', value)
  print()


  model.save_weights('./model/masking/DDI_masking_10fold'+str(i+1)+'.h5')

  result = np.array(baseline_results)
  results.append(result)



print("******************************************************************")
# print(np.array(results))

print(f"-Accuracy score_mean:{np.mean(np.array(results)[:,[1]])}")
# print(f"-Accuracy score_mean:{np.array(results)[:,[1]]}")
print(f"-Accuracy score_mean:{np.std(np.array(results)[:,[1]])}")

print(f"-Precision score_mean:{np.mean(np.array(results)[:,[2]])}")
print(f"-Precision score_mean:{np.std(np.array(results)[:,[2]])}")

print(f"-Recall score_mean:{np.mean(np.array(results)[:,[3]])}")
print(f"-Recall score_mean:{np.std(np.array(results)[:,[3]])}")

print(f"-AUC score_mean:{np.mean(np.array(results)[:,[4]])}")
print(f"-AUC score_mean:{np.std(np.array(results)[:,[4]])}")

print(f"-AUPR score_mean:{np.mean(np.array(results)[:,[5]])}")
print(f"-AUPR score_mean:{np.std(np.array(results)[:,[5]])}")






    
# 运行计时部分
time_end = time.time()
print(f"total running time: {(time_end - time_start)/60} minites")

(2248, 300)
***********fold-1***********
Epoch 1/300
15/15 [==============================] - 14s 744ms/step - loss: 0.6237 - precision: 0.7428 - recall: 0.7718 - accuracy: 0.6984 - auc: 0.4899 - prc: 0.6030 - val_loss: 0.5326 - val_precision: 0.7707 - val_recall: 0.8526 - val_accuracy: 0.7583 - val_auc: 0.5137 - val_prc: 0.5909 - lr: 0.0010
Epoch 2/300
15/15 [==============================] - 10s 681ms/step - loss: 0.5174 - precision: 0.7541 - recall: 0.8845 - accuracy: 0.7541 - auc: 0.5025 - prc: 0.6078 - val_loss: 0.5123 - val_precision: 0.7864 - val_recall: 0.8217 - val_accuracy: 0.7581 - val_auc: 0.5375 - val_prc: 0.6124 - lr: 0.0010
Epoch 3/300
15/15 [==============================] - 10s 680ms/step - loss: 0.5063 - precision: 0.7575 - recall: 0.8850 - accuracy: 0.7576 - auc: 0.5351 - prc: 0.6211 - val_loss: 0.4985 - val_precision: 0.7565 - val_recall: 0.8834 - val_accuracy: 0.7584 - val_auc: 0.5984 - val_prc: 0.6547 - lr: 0.0010
Epoch 4/300
15/15 [==============================]

In [6]:
from tensorflow.keras.callbacks import EarlyStopping#回调函数

# 定义回调函数：提前终止训练
earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0.0001, 
                          patience = 30,
                          verbose = 3,
                          mode = 'min',
                          restore_best_weights = True)


In [15]:
#加载模型
from keras.models import load_model
# model.load_weights(‘model_weights.h5')
model2 = create_model_DDI()

model2.build(k_X_train.shape)#加这一步有了解决

model2.load_weights('./model/masking/DDI_masking_10fold5.h5')

opt = Adam(learning_rate=0.001)
model2.compile(loss='binary_crossentropy', optimizer=opt, metrics=METRICS)
model2.fit(k_X_train,k_Y_train,batch_size=128, epochs=300,validation_data=(k_x_test,k_y_test),callbacks=[earlystop])
model2.evaluate(k_x_test,k_y_test,return_dict=True)

Epoch 1/300
16/16 [==============================] - 17s 861ms/step - loss: 0.4506 - precision: 0.8048 - recall: 0.8672 - accuracy: 0.7909 - auc: 0.7036 - prc: 0.7197 - val_loss: 0.4481 - val_precision: 0.8255 - val_recall: 0.8531 - val_accuracy: 0.7909 - val_auc: 0.6954 - val_prc: 0.7182
Epoch 2/300
16/16 [==============================] - 12s 778ms/step - loss: 0.4417 - precision: 0.8121 - recall: 0.8656 - accuracy: 0.7965 - auc: 0.7283 - prc: 0.7286 - val_loss: 0.4446 - val_precision: 0.8242 - val_recall: 0.8565 - val_accuracy: 0.7916 - val_auc: 0.6922 - val_prc: 0.7192
Epoch 3/300
16/16 [==============================] - 12s 769ms/step - loss: 0.4397 - precision: 0.8080 - recall: 0.8735 - accuracy: 0.7969 - auc: 0.7323 - prc: 0.7333 - val_loss: 0.4470 - val_precision: 0.8263 - val_recall: 0.8603 - val_accuracy: 0.7952 - val_auc: 0.6955 - val_prc: 0.7179
Epoch 4/300
16/16 [==============================] - 12s 761ms/step - loss: 0.4379 - precision: 0.8117 - recall: 0.8681 - accuracy

{'accuracy': 0.7983539700508118,
 'auc': 0.7114117741584778,
 'loss': 0.4361085295677185,
 'prc': 0.7283821105957031,
 'precision': 0.8234291076660156,
 'recall': 0.8711672425270081}

In [1]:
y_pre = model2.predict(k_x_test)
np.save('./model/masking/masking_DDI_pre.npy',y_pre)

NameError: ignored